In [ ]:
#Lymphoma Image Classifier with CNN

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Set random seed for reproducibility
import tensorflow as tf
import random
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

# === Step 1: Dataset Path ===
dataset_dir = "D:/RUTA/SEM 7/MEGA PROJECT/images/Lymphoma"

# === Step 2: Image Parameters ===
img_size = (128, 128)
batch_size = 32
epochs = 10

In [2]:
# === Step 3: Data Preprocessing ===
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

num_classes = len(train_gen.class_indices)

Found 12000 images belonging to 3 classes.
Found 3000 images belonging to 3 classes.


In [3]:
# === Step 4: CNN Model ===
model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

c:\Users\Ruta\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# === Step 5: Compile Model ===
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# === Step 6: Train Model ===
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint("lymphoma_model.h5", save_best_only=True)
]

history = model.fit(
    train_gen,
    steps_per_epoch = 50,
    validation_data=val_gen,
    validation_steps = 20,
    epochs=epochs,
    callbacks=callbacks
)

In [ ]:
# === Step 7: Evaluate Model ===
val_preds = model.predict(val_gen)
val_preds_labels = np.argmax(val_preds, axis=1)
true_labels = val_gen.classes
class_names = list(val_gen.class_indices.keys())

# Classification report
print("Classification Report:\n")
print(classification_report(true_labels, val_preds_labels, target_names=class_names))

# Confusion Matrix
print("Confusion Matrix:\n")
print(confusion_matrix(true_labels, val_preds_labels))

# Optional: ROC-AUC (only valid for binary or multilabel classification)
if num_classes == 2:
    roc_auc = roc_auc_score(true_labels, val_preds[:, 1])
    print(f"ROC AUC Score: {roc_auc:.4f}")

In [ ]:
# === Step 8: Save Final Model ===
model.save("lymphoma_classifier_final.h5")

In [ ]:
# === Step 9: Plot Training History ===
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Train Acc")
plt.plot(history.history["val_accuracy"], label="Val Acc")
plt.title("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.title("Loss")
plt.legend()
plt.tight_layout()
plt.show()